<a href="https://colab.research.google.com/github/gneusch/CovidDeathHungary/blob/main/logit_model/copy_teach_hits_to_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
colab = True

if (colab):
    from google.colab import drive
    drive.mount('/content/drive')
    from google.colab import files

import pandas as pd
import re
import string

import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
nltk.download('stopwords')
nltk.download('punkt')
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
stop_words = set(stopwords.words('english'))

Mounted at /content/drive
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
teach_filename = 'teaching_set.csv'
test_filename = 'testing_set.csv'

teach_data = pd.DataFrame()
test_data = pd.DataFrame()

if (colab): 
    teach_data = pd.read_csv(f'drive/My Drive/postings_201910/{teach_filename}')
    test_data = pd.read_csv(f'drive/My Drive/postings_201910/{test_filename}')
else:
    teach_data = pd.read_csv(f'./results/logistic_regression_model/5n/{teach_filename}')
    test_data = pd.read_csv(f'./results/logistic_regression_model/5n/{test_filename}')

skills = []

skill_file = 'drive/My Drive/postings_201910/all_ict_skill_labels.csv' if colab else './skill_dict/all_ict_skill_labels.csv'
        
with open(skill_file) as file:
    lines = file.readlines()
    
for skill in [skill.strip() for skill in lines]:
    skills.append(' '.join([w for w in word_tokenize(skill) if not w.lower() in stop_words]))

print(len(skills))

2110


In [ ]:
class PostingResultsWHits(dict):
  def __missing__(self, key):
    return False

hits_from_teaching_set = PostingResultsWHits()

for i in teach_data.index:
  rowData = []
  for j in teach_data[['resultfromposting','hit']]:
    rowData.append(teach_data.loc[i,(j)])

  if (not hits_from_teaching_set[rowData[0]]):
    hits_from_teaching_set[rowData[0]] = rowData[1]
  else:
    if (hits_from_teaching_set[rowData[0]] != rowData[1]):
      print(f'Two different hit value found for the phrase: {rowData[0]}. In dict: {hits_from_teaching_set[rowData[0]]}')

hits_from_teaching_set['information security data'] = 0

Two different hit value found for the phrase: information security data. In dict: 1
Two different hit value found for the phrase: information security data. In dict: 1
Two different hit value found for the phrase: information security data. In dict: 1
Two different hit value found for the phrase: support information security. In dict: 1
Two different hit value found for the phrase: information security data. In dict: 1
Two different hit value found for the phrase: information security data. In dict: 1
Two different hit value found for the phrase: information security data. In dict: 1
Two different hit value found for the phrase: information security data. In dict: 1
Two different hit value found for the phrase: information security data. In dict: 1
Two different hit value found for the phrase: information security data. In dict: 1
Two different hit value found for the phrase: information security data. In dict: 1


In [ ]:
for key, value in hits_from_teaching_set.items():    
  result = test_data.loc[test_data['result from posting'] == key]
  if not result.empty:
    rows = test_data.loc[test_data['result from posting'] == key]
    for index, row in rows.iterrows():
      hit_value = value if (value ==  row["hit"]) else -1
      test_data.at[index, 'manual_label'] = hit_value      

test_data['exact_dict_match'] = 0

for skill in skills:
  result = test_data.loc[test_data['result from posting'] == skill]
  if not result.empty:
    rows = test_data.loc[test_data['result from posting'] == skill]
    for index, row in rows.iterrows():
      test_data.at[index, 'exact_dict_match'] = 1
      if (test_data.at[index, 'manual_label'] != 1 | test_data.at[index, 'hit'] != 1):      
        test_data.at[index, 'manual_label'] = -1

test_data.loc[(test_data['manual_label'] != -1) & (test_data['hit'] == 1) & (test_data['Jaccard nts'] == 1), "manual_label"] = 1

test_data.head()

,Unnamed: 0,index,id,title,surrounding sentence,skill from dict,result from posting,N skill,N posting,Jaccard nts,Cosine nts,JaroWinkler nts,probability,hit,manual_label,exact_dict_match
0,0,0,7e65e2ceb05106c6,Integration/Configuration Suite Engineer,This role will be within the existing Integrat...,broadcast devices,devices,2,1,0.5,0.641689,0.518908,0.000046,0,NaN,0
1,1,1,7e65e2ceb05106c6,Integration/Configuration Suite Engineer,This role will be within the existing Integrat...,broadcast equipment,equipment,2,1,0.5,0.688247,0.275828,0.000042,0,NaN,0
2,2,2,7e65e2ceb05106c6,Integration/Configuration Suite Engineer,This role will be within the existing Integrat...,broadcasting equipment,equipment,2,1,0.5,0.639602,0.458754,0.000042,0,NaN,0
3,3,3,7e65e2ceb05106c6,Integration/Configuration Suite Engineer,This role will be within the existing Integrat...,cloud services,services,2,1,0.5,0.755929,0.732143,0.000109,0,0.0,0
4,4,4,7e65e2ceb05106c6,Integration/Configuration Suite Engineer,This role will be within the existing Integrat...,control devices,devices,2,1,0.5,0.683130,0.473016,0.000053,0,NaN,0


In [ ]:
if (colab): 
    test_data.to_csv('drive/My Drive/postings_201910/testing_set_manual_label.csv', sep=',', index=False)
else:
    test_data.to_csv('./results/logistic_regression_model/5n/testing_set_manual_label.csv', sep=',', index=False)
